#### Search Engine with Tools and Agents

In [2]:
print("Jay Ganesh")

Jay Ganesh


In [3]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper



In [4]:
api_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
##Now to call this API we need WikipediaQueryRun thing
wiki=WikipediaQueryRun(api_wrapper=api_wiki)
wiki.name

'wikipedia'

In [5]:
api_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=230)
arxiv=ArxivQueryRun(api_wrapper=api_arxiv)
arxiv.name

'arxiv'

In [6]:
tools=[wiki,arxiv]
## Above is the tool list

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI")
embeddings=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
vectorstoredb=FAISS.from_documents(documents,embeddings)
retriever=vectorstoredb.as_retriever()

In [19]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000215D034B9A0>, search_kwargs={})

In [58]:
from langchain_core.tools import create_retriever_tool
retrievertool=create_retriever_tool(retriever,"Langsmith-search","It search any info about the langsmith home page")

In [59]:
retrievertool.name

'Langsmith-search'

In [23]:
tools.append(retrievertool)

In [24]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Jay\\anaconda3\\envs\\ch25\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=230)),
 StructuredTool(name='kangsmith-search', description='It search any info about the langsmith home page', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x00000215D04CF520>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x00000215D052B250>)]

In [94]:
### Running all this tools with Agents and LLM Models
from langchain_groq import ChatGroq
grop_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(api_key=grop_api_key,model="llama-3.1-8b-instant",streaming=False)

In [95]:
from langchain_classic import hub
prompt=hub.pull("hwchase17/react")
# sys_prompts=prompt.messages[0].prompt.template
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [96]:
from langchain_classic.agents import create_react_agent  # Fixed path
agent = create_react_agent(llm, tools,prompt)
# result = agent.invoke({"input": "Tell me something about LangSmith?"}) 
# print(result["messages"][-1].content)


In [105]:
##Agent Executer
from langchain_classic.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [98]:
agent_executor

AgentExecutor(verbose=False, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nkangsmith-search(query: 'str', callbacks: 'Callbacks' = None) -> 'str | tuple[str, list[Document]]' - It search any info about the langsmith home page", 'tool_names': 'wikipedia, arxiv, kangsmith-search'}, metadata={'lc_h

In [107]:
agent_executor.invoke({"input":"Tell me something about langsmith?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about langsmith, it seems like a specific topic.
Action: kangsmith-search
Action Input: 'langsmith'It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place. LangSmith works with any agent stack, whether you’re using an existing framework or building your own. Prototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI agents.
​Get started
Create an accountSign up at smith.langchain.com (no credit card required).
You can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.
Copy the key and save it securely.Choose your integrationLangSmith works with many frameworks and providers including OpenAI, Anthropic, CrewAI, Vercel AI SDK, Pydantic AI, and more.
Browse available integrations to connect your stack.
Once your account and API key are ready, choose a quickstart to b

{'input': 'Tell me something about langsmith?',
 'output': 'LangSmith is not a well-known platform, and it appears to be a product or feature of the LangChain software framework, which is a software framework that helps facilitate the integration of large language models (LLMs) into applications.'}

In [108]:
agent_executor.invoke({"input":"What is Machine Learning?"})



> Entering new AgentExecutor chain...
Thought: I need to find a definition for Machine Learning, so I can start building my answer.

Action: wikipedia
Action Input: Machine LearningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genQuestion: What is Machine Learning?
Thought: I need to find a definition for Machine Learning, so I can start building my answer.
Action: wikipedia
Action Input: Machine LearningPage: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genQuestion: What is Machine Learning?
Thought: I need to find a definition for Machine Learning, so I can start building my answer.
Action: wikipedia
Action Input: Machine LearningPage: Machine learning
Summary: Machine learning (ML) is a fi

{'input': 'What is Machine Learning?',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate useful models.'}

In [ ]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...
Thought: The question is about a specific paper on arxiv.org, and we can use the arxiv tool to find the information.
Action: arxiv
Action Input: 1706.03762Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction mQuestion: What's the paper 1706.03762 about?
Thought: The question is about a specific paper on arxiv.org, and we can use the arxiv tool to find the information.
Action: arxiv
Action Input: 1706.03762Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction mQuestion: What's the paper 1706.03762 about?
Thought: The question is about a specific paper on arxiv.org, and we can use the arxiv tool to find 